In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Hyper-parameters

In [27]:
n_epochs = 10
batch_size = 4
learning_rate = 0.001

In [28]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

## Set train and test datasets

In [29]:
train_dataset = torchvision.datasets.CIFAR10(
    root='./CIFAR10/data',
    train=True,
    download=True,
    transform=transform
)

test_dataset = torchvision.datasets.CIFAR10(
    root='./CIFAR10/data',
    train=False,
    download=True,
    transform=transform
)

Files already downloaded and verified
Files already downloaded and verified


## Dataloaders

In [30]:
train_dl = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dl = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [31]:
classes = ('plane', 'car', 'brid', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

## Model

In [32]:
# Input size is 3 because we will send 3 types of color channels
input_size = 3
output_size = 6
kernel_size = 5

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # Feature learning
        self.conv1 = nn.Conv2d(input_size, output_size, kernel_size)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(output_size, 16, kernel_size)
        # Classification
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.reshape(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


model = ConvNet()

## Loss function

In [33]:
# CrossEntropyLoss Includes the softmax
criterion = nn.CrossEntropyLoss()

## Optimizer

In [34]:
optimzer = torch.optim.SGD(model.parameters(), lr=learning_rate)

## Train model

In [35]:
n_total_steps = len(train_dl)

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_dl):
        # Forward
        output = model(images)

        # Loss
        loss = criterion(output, labels)

        # Backward
        optimzer.zero_grad()
        loss.backward()
        # Update the parameters
        optimzer.step()

        if (i+1) % 1000 == 0:
            print(f"epoch: {epoch+1}/{n_epochs}, step: {i+1}/{n_total_steps}, loss: {loss.item():.4f}")

epoch: 1/10, step: 1000/12500, loss: 2.2503
epoch: 1/10, step: 2000/12500, loss: 2.3187
epoch: 1/10, step: 3000/12500, loss: 2.3183
epoch: 1/10, step: 4000/12500, loss: 2.3092
epoch: 1/10, step: 5000/12500, loss: 2.3282
epoch: 1/10, step: 6000/12500, loss: 2.2937
epoch: 1/10, step: 7000/12500, loss: 2.3032
epoch: 1/10, step: 8000/12500, loss: 2.3040
epoch: 1/10, step: 9000/12500, loss: 2.3005
epoch: 1/10, step: 10000/12500, loss: 2.3101
epoch: 1/10, step: 11000/12500, loss: 2.3114
epoch: 1/10, step: 12000/12500, loss: 2.1060
epoch: 2/10, step: 1000/12500, loss: 2.2292
epoch: 2/10, step: 2000/12500, loss: 2.0540
epoch: 2/10, step: 3000/12500, loss: 2.2833
epoch: 2/10, step: 4000/12500, loss: 2.2092
epoch: 2/10, step: 5000/12500, loss: 2.1129
epoch: 2/10, step: 6000/12500, loss: 1.5029
epoch: 2/10, step: 7000/12500, loss: 1.8678
epoch: 2/10, step: 8000/12500, loss: 1.9691
epoch: 2/10, step: 9000/12500, loss: 1.6772
epoch: 2/10, step: 10000/12500, loss: 2.1661
epoch: 2/10, step: 11000/125

## Validate model

In [36]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]

    for images, labels in test_dl:
        output = model(images)

        # Return value and index 
        _, predictions = torch.max(output, 1)

        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

        for i in range(batch_size):
            label = labels[i]
            pred = predictions[i]
            if label == pred:
                n_class_correct[label] += 1
            n_class_samples[label] += 1
    
    acc = 100 * n_correct / n_samples
    print(f"Accuracy of CNN: {acc}")

    for i in range(10):
        acc = 100 * n_class_correct[i] / n_class_samples[i]
        print(f"Accuracy of {classes[i]}: {acc}")




Accuracy of CNN: 57.82
Accuracy of plane: 59.4
Accuracy of car: 69.6
Accuracy of brid: 41.4
Accuracy of cat: 42.9
Accuracy of deer: 35.0
Accuracy of dog: 59.0
Accuracy of frog: 76.8
Accuracy of horse: 57.7
Accuracy of ship: 73.8
Accuracy of truck: 62.6
